#  [Alpha Version] Tourist Advisory Expert System

## Importing the Libraries

In [1]:
import pandas as pd
from sodapy import Socrata
import tempfile
import os
import csv

from pyswip.prolog import Prolog
from pyswip.easy import *
import time
import copy
import math
import random

## Data Processing & Generation

### Important note: Make sure to unzip 'city_temperature.csv.zip'

In [2]:
#Importing the European Captials
european_capitals = []
with open('input_cities.csv') as csv_file:
    reader = csv.reader(csv_file)
    next(reader)
    for row in reader:
        european_capitals.append(row[1])
#Extracting Temperatures
world_tempratures = pd.read_csv('city_temperature.csv')

/Users/kareemalsayed/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
cities_available_data = world_tempratures['City'].unique()
cities_average_temps = {}
def convert_to_celsius(Fahrenheit):
    return (Fahrenheit - 32) * 5.0/9.0
mon_number = {  1:'January',
                2:'February',
                3:'March',
                4:'April',
                5:'May', 
                6:'June', 
                7:'July',
                8:'August',
                9:'September',
                10:'October',
                11:'November',
                12:'December'}

empty_dict_months = {'January': None,
                    'February': None,
                    'March': None,
                    'April': None,
                    'May': None, 
                    'June': None, 
                    'July': None,
                    'August': None,
                    'September': None,
                    'October': None,
                    'November': None,
                    'December': None}

# Filtering out cities with missing data
for city in european_capitals:
    if city in cities_available_data:
        cities_average_temps[city] = copy.deepcopy(empty_dict_months)

In [4]:
# Calculating data per month
for city in cities_average_temps:
    years = world_tempratures.loc[(world_tempratures['City'] == city)]['Year'].unique()
    avg_months = {}
    for year in years:
        for month in range(1, 13):
            avg_temp = world_tempratures.loc[(world_tempratures['City'] == city)
                                             &(world_tempratures['Month'] == month)
                                             &(world_tempratures['Year'] == year)]['AvgTemperature'].mean()
            if avg_temp != -99 and not math.isnan(avg_temp):
                if mon_number[month] in avg_months:
                    avg_months[mon_number[month]].append(avg_temp)
                else:
                    avg_months[mon_number[month]] = [avg_temp]
    for month in avg_months:
        cities_average_temps[city][month] = convert_to_celsius(int(sum(avg_months[month])/len(avg_months[month])))
    avg_months = {}

In [130]:
# Generate Prolog Scripts for temperatures
tempratures_prolog = ""
for city in cities_average_temps:
    for month in cities_average_temps[city]:
        if cities_average_temps[city][month] >= 23:
            tempratures_prolog += f"temperature({city.lower()}, high, {month.lower()})."
        elif cities_average_temps[city][month] >= 14:
            tempratures_prolog += f"temperature({city.lower()}, moderate, {month.lower()})."
        else:
            tempratures_prolog += f"temperature({city.lower()}, low, {month.lower()})."
        tempratures_prolog += '\n'
        
# Creating dummy precipitation data for cities
random.seed (15)
precipitation_prolog = ""
for city in cities_average_temps:
    random_pick = random.random()
    if random_pick > 0.5:
        precipitation_prolog += f"precipitation({city.lower()}, high)."
    else:
        precipitation_prolog += f"precipitation({city.lower()}, low)."
    precipitation_prolog += '\n'

## Messages

In [136]:
msg_welcome = """

 __       __            __                                                    __ 
|  \  _  |  \          |  \                                                  |  \
| $$ / \ | $$  ______  | $$  _______   ______   ______ ____    ______        | $$
| $$/  $\| $$ /      \ | $$ /       \ /      \ |      \    \  /      \       | $$
| $$  $$$\ $$|  $$$$$$\| $$|  $$$$$$$|  $$$$$$\| $$$$$$\$$$$\|  $$$$$$\      | $$
| $$ $$\$$\$$| $$    $$| $$| $$      | $$  | $$| $$ | $$ | $$| $$    $$       \$$
| $$$$  \$$$$| $$$$$$$$| $$| $$_____ | $$__/ $$| $$ | $$ | $$| $$$$$$$$       __ 
| $$$    \$$$ \$$     \| $$ \$$     \ \$$    $$| $$ | $$ | $$ \$$     \      |  \
 \$$      \$$  \$$$$$$$ \$$  \$$$$$$$  \$$$$$$  \$$  \$$  \$$  \$$$$$$$       \$$ \n

This Program will help you to get a customized recommendation for city to have an amazing trip!
"""
msg_rain = """

, // ,,/ ,.// ,/ ,// / /, // ,/, /, // ,/,
/, // ,/,_|_// ,/ ,, ,/, // ,/ /, //, /,/
 /, /,.-'   '-. ,// ////, // ,/,/, // ///
, ,/,/         \ // ,,///, // ,/,/, // ,
,/ , ^^^^^|^^^^^ ,// ///  /,,/,/, ///, //
 / //     |  O    , // ,/, //, ///, // ,/
,/ ,,     J\/|\_ |+'(` , |) ^ ||\|||\|/` |
 /,/         |   || ,)// |\/-\|| ||| |\] .
/ /,,       /|    . ,  ///, . /, // ,//, /
, /        \ \    ). //, ,( ,/,/, // ,/,

Do you prefer rainy weather?

1: I don't mind rain
2: I prefer dry weather

"""

msg_temperature = '''
              .     :     .
            .  :    |    :  .
             .  |   |   |  ,
              \  |     |  /
          .     ,-'"""`-.     .
            "- /  __ __  \ -"
              |==|  I  |==|
        - --- | _`--^--'_ | --- -
              |'`.     ,'`|
            _- \  "---"  / -_
          .     `-.___,-'     .
              /  |     |  \
            .'  |   |   |  `.
               :    |    :
              .     :     .
              
What temperature levels do you prefer? 

1: Cold Temperature (T < 15°C)
2: Moderate Temperature (15°C < T < 25°C )
3: High Temperature (25°C < T)
'''

msg_outdoor = """

                   \  |  /         ___________
    ____________  \ \_# /         |  ___      |       _________
   |            |  \  #/          | |   |     |      | = = = = |
   | |   |   |  |   \\#           | |`v'|     |      |         |
   |            |    \#  //       |  --- ___  |      | |  || | |
   | |   |   |  |     #_//        |     |   | |      |         |
   |            |  \\ #_/_______  |     |   | |      | |  || | |
   | |   |   |  |   \\# /_____/ \ |      ---  |      |         |
   |            |    \# |+ ++|  | |  |~~~~~~| |      | |  || | |
   |            |    \# |+ ++|  | |  |~~~~~~| |      | |  || | |
 ~~|    (~~~~~) |~~~~~#~| H  |_ |~|  | |||| | |~~~~~~|         |
   |    ( ||| ) |     # ~~~~~~    |  | |||| | |      | ||||||| |
   ~~~~~~~~~~~~~________/  /_____ |  | |||| | |      | ||||||| |
                                  ~~~~~~~~~~~~~      | ||||||| |
How long are you planning to spend in outdoor areas?

1: Little time
2: Long time
"""

msg_month = """
                                                    
            ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒                            
            ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒                            
            ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                            
            ░░░░░░░░██░░░░██░░░░██░░░░██░░                            
            ░░░░░░░░██░░██░░██░░████░░██░░                            
            ░░░░░░░░██░░██░░██░░████████░░                            
            ░░██░░░░██░░██████░░██░░████░░                            
            ░░░░████░░░░██░░██░░██░░░░██░░                            
            ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                            
            ░░░░░░░░░░░░░░▒▒▒▒░░░░░░░░░░░░                            
            ░░░░░░░░░░░░▒▒▒▒▒▒░░░░░░░░░░░░                            
            ░░░░░░░░░░░░░░▒▒▒▒░░░░░░░░░░░░                            
            ░░░░░░░░░░░░░░▒▒▒▒░░░░░░░░░░░░                            
            ░░░░░░░░░░░░░░▒▒▒▒░░░░░░░░░░░░                            
            ░░░░░░░░░░░░░░▒▒▒▒░░░░░░░░░░░░                            
            ░░░░░░░░░░░░▒▒▒▒▒▒▒▒░░░░░░░░░░                            
            ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                            
                                                                                        
What month are you planning to travel?

please enter a month between 1 - 12
"""

## KB

In [142]:
KB = """
%  Tell prolog that known/3 will be added later by asserta
:- dynamic known/3.

% Precipitation Information\n 
""" + precipitation_prolog + """
% Temperature Information\n 
"""+tempratures_prolog +"""

% The code below implements the prompting to ask the user:
preferrain(X) :- ask(preferrain, X).
prefertemperature(X) :- ask(prefertemperature, X).
stayoutdoors(X) :- ask(stayoutdoors, X).
travelmonth(X) :- ask(travelmonth, X).

% Climate Assessments
suitable(C, outdoor) :- 
    stayoutdoors(short); 
    (precipitation(C, high), preferrain(any));
    (precipitation(C, low), (preferrain(any) ; preferrain(low))).
suitable(C, temperature) :- temperature(C, L, M), prefertemperature(L), travelmonth(M).
suitable(C, climate) :- suitable(C, temperature), suitable(C, outdoor).

% Overall City Assessments
recommend(C) :- suitable(C, climate).


% Asking clauses
multivalued(none).



ask(A, V):-
known(yes, A, V), % succeed if true
!.	% stop looking

ask(A, V):-
known(_, A, V), % fail if false
!, 
fail.

% If not multivalued, and already known, don't ask again for a different value.
ask(A, V):-
\+multivalued(A),
known(yes, A, V2),
V \== V2,
!.

ask(A, V):-
read_py(A,V,Y), % get the answer
asserta(known(Y, A, V)), % remember it
Y == yes.	% succeed or fail

ask(A, V):-
 not multivalued(A),
 known(yes, A, V2),
 V \== V2,
 !,
 fail.
 
"""


prolog = Prolog() # Global handle to interpreter

retractall = Functor("retractall")
known = Functor("known",3)

prefer_rain_options = ['1', '2']
prefer_temp_options = ['1', '2', '3']
stay_outdoor_options = ['1', '2']
month_options = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']

def ask(message, key_words=['yes', 'no'], menu=False):
    """
    Asks the user a normal 'yes or no' question or a
    'menu' style question. Returns the response of the user.
    
    message: a string containing the message to display to user
    key_words: a list containing the responses that are allowed
    menu: boolean indicating if question is a menu question
    """
    valid_response = False 
    time.sleep (1)
    response = input(message).strip().lower() # Ensure no white spaces in response and covert to lower case
    return response


# Define foreign functions for getting user input and writing to the screen
def write_py(X):
    print(str(X))
    sys.stdout.flush()
    return True

def read_py(A,V,Y):
    global prefer_rain_response
    global prefer_temp_response
    global stay_outdoor_response
    global month_response

    if isinstance(Y, Variable):
        if str(A) == "stayoutdoors":
            if stay_outdoor_response is None:
                stay_outdoor_response = ask(msg_outdoor)
            if str(V) == 'short' and stay_outdoor_response == '1':
                Y.unify('yes')
            elif str(V) == 'long' and stay_outdoor_response == '2':
                Y.unify('yes')
            else:
                Y.unify('no')
        elif str(A) == "preferrain":
            if prefer_rain_response is None:
                prefer_rain_response = ask(msg_rain)
            if str(V) == 'any' and prefer_rain_response == '1':
                Y.unify('yes') 
            elif str(V) == 'low' and prefer_rain_response == '2':
                Y.unify('yes')
            else: 
                Y.unify('no')
        elif str(A) == "prefertemperature":
            if prefer_temp_response is None:
                prefer_temp_response = ask(msg_temperature)
            if str(V) == 'low' and prefer_temp_response == '1':
                Y.unify('yes') 
            elif str(V) == 'moderate' and prefer_temp_response == '2':
                Y.unify('yes')
            elif str(V) == 'high' and prefer_temp_response == '3':
                Y.unify('yes')
            else: 
                Y.unify('no')
        elif str(A) == "travelmonth":
            if month_response is None:
                month_response = ask(msg_month) 
            if mon_number[int(month_response)].lower() == str(V):
                Y.unify('yes')
            else:
                Y.unify('no')
        return True 
    else:
        return False

## Testing out the program

In [147]:
prefer_rain_response = None
prefer_temp_response = None
stay_outdoor_response = None
month_response = None


write_py.arity = 1
read_py.arity = 3

registerForeign(read_py)
registerForeign(write_py)

# Create a temporary file with the KB in it
(FD, name) = tempfile.mkstemp(suffix='.pl', text = "True")
with os.fdopen(FD, "w") as text_file:
    text_file.write(KB)
prolog.consult(name) # open the KB for consulting
os.unlink(name) # Remove the temporary file

call(retractall(known))

print(msg_welcome)
go_to = [s for s in prolog.query("recommend(City).", maxresult=1)]

if (len(go_to) != 0):
    city = go_to[0]['City']
    print(f"You can visit {city.capitalize()}!")
else:
    print("Sorry, we couldn't find a good match for you now!")



 __       __            __                                                    __ 
|  \  _  |  \          |  \                                                  |  | $$ / \ | $$  ______  | $$  _______   ______   ______ ____    ______        | $$
| $$/  $\| $$ /      \ | $$ /       \ /      \ |      \    \  /      \       | $$
| $$  $$$\ $$|  $$$$$$\| $$|  $$$$$$$|  $$$$$$\| $$$$$$\$$$$\|  $$$$$$\      | $$
| $$ $$\$$\$$| $$    $$| $$| $$      | $$  | $$| $$ | $$ | $$| $$    $$       \$$
| $$$$  \$$$$| $$$$$$$$| $$| $$_____ | $$__/ $$| $$ | $$ | $$| $$$$$$$$       __ 
| $$$    \$$$ \$$     \| $$ \$$     \ \$$    $$| $$ | $$ | $$ \$$     \      |   \$$      \$$  \$$$$$$$ \$$  \$$$$$$$  \$$$$$$  \$$  \$$  \$$  \$$$$$$$       \$$ 


This Program will help you to get a customized recommendation for city to have an amazing trip!


              .     :     .
            .  :    |    :  .
             .  |   |   |  ,
              \  |     |  /
          .     ,-'"""`-.     .
            "- /

## References
- https://www.kaggle.com/sudalairajkumar/daily-temperature-of-major-cities